In [16]:
# !pip install tensorflow==2.4.1
# !pip install pyarrow

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from math import ceil

2022-01-17 17:21:55.006465: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-17 17:21:55.006493: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def check_targets(targs):
    if targs[0] == -1:
        return 1
    else:
        return 0

In [3]:
def cut_length(data, seq_len=512):
    return data[:seq_len]

In [4]:
def create_tfrecords_dataset(data, iter_num, dataset_type='train'):
    # Removing samples that have no targets
    data['no_target'] = data['target_tok'].apply(check_targets)
    data = data[data['no_target']==0].copy()
    
    # Cutting the length of the abstracts and paper titles
    data['paper_title_tok'] = data['paper_title_tok'].apply(cut_length, args=(32,))
    data['abstract_tok'] = data['abstract_tok'].apply(cut_length, args=(256,))
    
    paper_title = tf.ragged.constant(data['paper_title_tok'].to_list())
    abstract = tf.ragged.constant(data['abstract_tok'].to_list())
    
    # Padding the targets to be easily fed into the TF model
    targets = tf.keras.preprocessing.sequence.pad_sequences(data['target_tok'].to_list(), maxlen=15, 
                                                            dtype='int64', padding='post', 
                                                            truncating='post', value=0)

    ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(paper_title),
                              tf.data.Dataset.from_tensor_slices(abstract),
                              tf.data.Dataset.from_tensor_slices(data['journal_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(data['doc_type_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(targets)))
    
    serialized_features_dataset = ds.map(tf_serialize_example)
    
    filename = f"./data/iteration_2/tfrecords/{dataset_type}/{str(iter_num).zfill(4)}.tfrecord"
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_features_dataset)

In [5]:
def tf_serialize_example(f0, f1, f2, f3, f4):
    tf_string = tf.py_function(serialize_example, (f0, f1, f2, f3, f4), tf.string)
    return tf.reshape(tf_string, ())

In [6]:
def serialize_example(paper_title, abstract, journal, doc_type, targets):
    paper_title_list = tf.train.Int64List(value=paper_title.numpy().tolist())
    abstract_list = tf.train.Int64List(value=abstract.numpy().tolist())
    journal_list = tf.train.Int64List(value=journal.numpy().tolist())
    doc_type_list = tf.train.Int64List(value=doc_type.numpy().tolist())
    targets_list = tf.train.Int64List(value=targets.numpy().tolist())
    
    paper_title_feature = tf.train.Feature(int64_list = paper_title_list)
    abstract_feature = tf.train.Feature(int64_list = abstract_list)
    journal_feature = tf.train.Feature(int64_list = journal_list)
    doc_type_feature = tf.train.Feature(int64_list = doc_type_list)
    targets_feature = tf.train.Feature(int64_list = targets_list)
    
    features_for_example = {
        'paper_title': paper_title_feature,
        'abstract': abstract_feature,
        'journal': journal_feature,
        'doc_type': doc_type_feature,
        'targets': targets_feature
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=features_for_example))
    
    return example_proto.SerializeToString()

In [10]:
def turn_part_file_into_tfrecord(base_path, dataset_type='train'):
    file_list = [x for x in os.listdir(f"{base_path}{dataset_type}") if x.endswith('parquet')]
    file_list.sort()
    print(f"There are {len(file_list)} files for {dataset_type}")
    for i, file_name in enumerate(file_list):
        data = pd.read_parquet(f"{base_path}{dataset_type}/{file_name}")
        print(f"_____File number: {i} ({data.shape[0]} samples)")
        create_tfrecords_dataset(data, i, dataset_type)

In [11]:
base_file_path = f"./data/iteration_1/tokenized_data/"

In [12]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'train')

There are 100 files for train
_____File number: 0 (1486621 samples)


2022-01-17 17:53:53.717150: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-17 17:53:53.718864: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-17 17:53:53.718872: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-17 17:53:53.718887: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-31-169.ec2.internal): /proc/driver/nvidia/version does not exist
2022-01-17 17:53:53.719968: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebui

_____File number: 1 (991672 samples)
_____File number: 2 (1486468 samples)
_____File number: 3 (1486876 samples)
_____File number: 4 (1487474 samples)
_____File number: 5 (990006 samples)
_____File number: 6 (1487822 samples)
_____File number: 7 (991953 samples)
_____File number: 8 (993361 samples)
_____File number: 9 (991922 samples)
_____File number: 10 (991206 samples)
_____File number: 11 (992267 samples)
_____File number: 12 (1486685 samples)
_____File number: 13 (991641 samples)
_____File number: 14 (992454 samples)
_____File number: 15 (991873 samples)
_____File number: 16 (992159 samples)
_____File number: 17 (1488779 samples)
_____File number: 18 (989652 samples)
_____File number: 19 (989146 samples)
CPU times: user 2h 38min 12s, sys: 7min 31s, total: 2h 45min 44s
Wall time: 2h 27min 45s


In [ ]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'val')

There are 10 files for val
_____File number: 0 (62547 samples)
_____File number: 1 (24745 samples)
_____File number: 2 (69705 samples)
_____File number: 3 (60810 samples)
_____File number: 4 (26665 samples)


In [ ]:
# %%time
# turn_part_file_into_tfrecord(base_file_path, 'test')